<img align="left" src="https://lever-client-logos.s3.amazonaws.com/864372b1-534c-480e-acd5-9711f850815c-1524247202159.png" width=200>
<br></br>
<br></br>

# Neural Networks

## *Data Science Unit 4 Sprint 2 Assignment 1*

## Define the Following:
You can add image, diagrams, whatever you need to ensure that you understand the concepts below.

### Input Layer:
The first layer of an Artificial Neural Network (ANN) which takes the values of features as inputs.

### Hidden Layer:
Layers between the input and output layers that add complexity to the ANN.

### Output Layer:
The final layer of an ANN. This layer returns the "answer" we are interested in predicting.

### Neuron:
An individual node in an ANN. Each layer is made up of a group of Neurons.

### Weight:
A scalar that is assigned to a Neuron in the network.

### Activation Function:
A function that determines the output of a neuron in an ANN. Common activation functions are the sigmoid, tanh, and approximations of the Heaviside function.

### Node Map:
A visual representation of an ANN's structure. Circles represent neurons and data flows left to right (generally). Layers are grouped vertically.

### Perceptron:
The simplest ANN. A Perceptron contains only an input and output layer, no hidden layers.


## Inputs -> Outputs

### Explain the flow of information through a neural network from inputs to outputs. Be sure to include: inputs, weights, bias, and activation functions. How does it all flow from beginning to end?

#### The features from the dataset are sent to the input nodes that constitute the input layer. In each node of the input layer, the input value is weighted according to the neuron's unique weight, then processed by the activation function. The output of the activation function is then sent to every neuron in the next layer as input, and the process is repeated until the data reaches the output layer.

## Write your own perceptron code that can correctly classify a NAND gate. 

| x1 | x2 | y |
|----|----|---|
| 0  | 0  | 1 |
| 1  | 0  | 1 |
| 0  | 1  | 1 |
| 1  | 1  | 0 |

In [1]:
import numpy as np
import matplotlib.pyplot as plt

In [21]:
# Define inputs [x_1, x_2]
inputs = np.array([
    [0, 0],
    [1, 0],
    [0, 1],
    [1, 1]
])

# Correct outputs:
correct_output = np.array([
    [1],
    [1],
    [1],
    [0]
])

In [63]:
import math
# Activation Function
def sigmoid(x):
    return 1 / (1 + np.exp(-x))

# Derivative of sigmoid
def sigmoid_derivative(x):
    sx = sigmoid(x)
    return sx * (1-sx)

In [64]:
# Initialize random weights for x_1, x_2
# Keep weights bounded [-1, 1]
weights = 2 * np.random.random((2, 1)) - 1
weights

array([[ 0.88361341],
       [-0.85054223]])

In [65]:
# iterations = []
# errors = []
# outputs = []
# weight_list = []
# gradients = []

In [66]:
# Iterate over 10,000 Epochs
for iteration in range(10000):
    
    # Find the weighted sum
    weighted_sum = np.dot(inputs, weights)

    # Send weighted sum to activation function
    activated_output = sigmoid(weighted_sum)
#     outputs.append(activated_output)

    # Find the error
    error = correct_output - activated_output
#     iterations.append(iteration)
#     errors.append(error)

    # Calculate adjustments using the derivative of our activation function
    adjustments = error * sigmoid_derivative(activated_output)
#     gradients.append(sigmoid_derivative(activated_output))

    # Adjust weights
    weights = weights + np.dot(inputs.T, adjustments)
#     weight_list.append(weights)

In [67]:
print("Weights after training")
print(weights)

print ('')

print("Output after training")
print(activated_output)

Weights after training
[[ 1.38777878e-16]
 [-2.91433544e-16]]

Output after training
[[0.5]
 [0.5]
 [0.5]
 [0.5]]


## Implement your own Perceptron Class and use it to classify a binary dataset like: 
- [The Pima Indians Diabetes dataset](https://raw.githubusercontent.com/ryanleeallred/datasets/master/diabetes.csv) 
- [Titanic](https://raw.githubusercontent.com/ryanleeallred/datasets/master/titanic.csv)
- [A two-class version of the Iris dataset](https://raw.githubusercontent.com/ryanleeallred/datasets/master/Iris.csv)

You may need to search for other's implementations in order to get inspiration for your own. There are *lots* of perceptron implementations on the internet with varying levels of sophistication and complexity. Whatever your approach, make sure you understand **every** line of your implementation and what its purpose is.

In [9]:
import pandas as pd

from sklearn.model_selection import train_test_split

In [10]:
# Import Pima Indians Diabetes Dataset
df = pd.read_csv('https://raw.githubusercontent.com/ryanleeallred/datasets/master/diabetes.csv')

In [11]:
# Let's see what we're working with
print(df.shape)
df.head()

(768, 9)


,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


In [12]:
# Set features and targets
features = ['Pregnancies', 'Glucose', 'BloodPressure',
            'SkinThickness', 'Insulin', 'BMI', 'DiabetesPedigreeFunction',
            'Age']

target = 'Outcome'

In [13]:
# X and y
X = df[features]
y = df[target]

In [14]:
# Initialize weights
# Keep weights bounded [-1, 1]
weights = 2 * np.random.random((8, 1)) - 1
weights

array([[ 0.66098563],
       [ 0.8320193 ],
       [ 0.82456964],
       [-0.37272133],
       [ 0.42290192],
       [ 0.40423482],
       [ 0.29752069],
       [-0.67975676]])

In [82]:
class Perceptron():
    
    def fit(X, y):
        
        # Set weights
        weights = 2 * np.random.random((X.shape[1], 1)) - 1
        
        # Iterate over 10,000 Epochs
        for iteration in range(10000):
            
            # Find the weighted sum
            weighted_sum = np.dot(X.values, weights)

            # Send weighted sum to activation function
            activated_output = sigmoid(weighted_sum)

            # Find the error
            error = y.values - activated_output
        #     iterations.append(iteration)
        #     errors.append(error)

            # Calculate adjustments using the derivative of our activation function
            adjustments = error * sigmoid_derivative(activated_output)

            # Adjust weights
            weights = weights + np.dot(X.values.T, adjustments)
            
        # Return predictions
        return activated_output

In [83]:
results = Perceptron.fit(X, y)

/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:4: RuntimeWarning: overflow encountered in exp
  after removing the cwd from sys.path.


In [84]:
print("Output after training")
print(results)

Output after training
[[1. 0. 1. ... 0. 1. 0.]
 [1. 0. 1. ... 0. 1. 0.]
 [1. 0. 1. ... 0. 1. 0.]
 ...
 [1. 0. 1. ... 0. 1. 0.]
 [1. 0. 1. ... 0. 1. 0.]
 [1. 0. 1. ... 0. 1. 0.]]


In [85]:
# Check my error
final_error = ((results - y.values) ** 2).sum()
final_error

0.0

## Stretch Goals:

- Research "backpropagation" to learn how weights get updated in neural networks (tomorrow's lecture). 
- Implement a multi-layer perceptron. (for non-linearly separable classes)
- Try and implement your own backpropagation algorithm.
- What are the pros and cons of the different activation functions? How should you decide between them for the different layers of a neural network?